# Investigate Titanic Survivor

<img src='img/nytimes.jpg'>

This is a final project from online class in Udacity, [UD170 Intro to Data Analyst](https://classroom.udacity.com/courses/ud170/) to analyze titanic survivor by using [Titanic dataset](https://www.kaggle.com/c/titanic) on Kaggle Competition. This project focus on doing Exploratory Data Analysis (EDA) to extract insight on the dataset.

# <a name='Definition'>Problem Definition</a>

### Objective
- What factors made people more likely to survive?

### Description
>The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

>One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

## Import Libraries
We need to import python libraries that containing the necessary functionality we will need

In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import re

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

## Acquire Data

In [ ]:
dataset = pd.read_csv('data/titanic_data.csv')
dataset.head()

## Analyze by describing data

### Which features are available in the dataset?

Here's the feature description:

- <b>`Survived`</b>: Outcome of survival
- <b>`Pclass`</b>: Socio-economic class
- <b>`Name`</b>: Name of passenger
- <b>`Sex`</b>: Sex of the passenger
- <b>`Age`</b>: Age of the passenger
- <b>`SibSp`</b>: Number of siblings and spouses of the passenger aboard
- <b>`Parch`</b>: Number of parents and children of the passenger aboard
- <b>`Ticket`</b>: Ticket number of the passenger
- <b>`Fare`</b>: Fare paid by the passenger
- <b>`Cabin`</b>: Cabin number of the passenger
- <b>`Embarked`</b>: Port of embarkation of the passenger 

For more information, you can see [this documentation.](https://www.kaggle.com/c/titanic/data)

### Which features are categorical?
Categorical represent types of data which may be divided into group. Example of categorical group are race, sex, age group, and educational level. Althought the latter two group may also be considered in a numerical manner, it's often more informative to categorize into small number of groups.
- `Survived` (True or False / 1 or 0)
- `Sex` (Male, Female) 
- `Embarked` (C, S, Q)
- `Pclass` (1, 2, 3)

### Which features are numerical?
Numerical represent types of data which can be measurement, such as a person's height, weight, IQ, or blood pressure. It can be discrete or continous.
- `Age` (continous)
- `Fare` (continous)
- `Sibsp` (discrete)
- `Parch` (discrete)

### Which features are mixed data types?
Numerical, alphanumeric data within same feature. These are candidates for correcting.
- `Ticket` is a mix of numeric and alphanumeric
- `Cabin` is alphanumeric

### Which features may contain errors or typos?
This is harder to review for a large dataset, however reviewing a few samples from a smaller dataset may just tell us outright, which features may require correcting.
- `Name` feature may contain error of typos as there are several ways used to describe a name including titles, round brackets, and quotes used for alternative or short names

### Which features contain blank, null or empty values?
These will require correcting before used for analyzing.
- `Cabin` : 687 records
- `Age` : 177 records
- `Embarked` : 2 records

In [ ]:
total_null = dataset.isnull().sum()
percent_null = (round(total_null/dataset.isnull().count()*100, 2))
missing_data = pd.concat([total_null, percent_null], axis=1, keys=['Total', '%']).sort_values('%', ascending=False)
missing_data.head()

`Embarked` featue has only 2 missing values, which can easily to handled. To deal with `Age` which has 117 missing feature, it'll be much more tricky. The `Cabin` feature needs further investigation, but it looks like that we might want to drop from the dataset since 77% of missing value

### What is the distribution of numerical feature values across the samples?
This helps us determine, among other early insights, how representative is the dataset of the actual problem domain.
- Total samples are 891 or 40% of actual number of passenger on board the titanic (2,224)
- There are 38% survived representasive of actual survival rate at 32%
- Few elderly passenger (<1) within age range 65-80
- Fares varied significantly with few passengers (<1%) paying as high as $512.
- Most passengers (75%) did not travel with parent and child
- Nearly 33% passengers had siblings and/or spouse aboard

In [ ]:
dataset.describe()
# age and fare : percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .99]
# Parch : percentiles=[.75]
# SibSp : percentiles=[.68]

### What is the distribution of categorical features?
This helps us determine, among other early insights, how representative is the dataset of the actual problem domain.
- All name of passengers are unique (count=891)
- `Sex` feature have two possibles values with male is the most passengers (count=577/891)
- `Ticket` has high ratio (22%) of duplicate values
- `Cabin` also has duplicate values. That might be several passenger shared a cabin
- There are three possible of `Embarked` values (S=Southampton, Q=Queenstown, C=Cherbourg)

In [ ]:
dataset.describe(include=['O'])

## Analyzing by data visualization

### `Age` and `Sex`

In [ ]:
survived_sex = dataset[dataset['Survived'] == 1]['Sex'].value_counts()
dead_sex = dataset[dataset['Survived'] == 0]['Sex'].value_counts()
df_sex = pd.DataFrame([survived_sex, dead_sex], index=['Survived', 'Dead'])

# using pandas.dataframe.plot
df_sex.plot(kind='bar', stacked=True, figsize=(10,5), title='Distribution survival based on gender')

The `Sex` feature seems to be decisive feature. Women are more likely to survive

In [ ]:
# create label
survived = 'Survived'
not_survived = 'Not Survived'

# initalize figure and axes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 7), sharey=True)

female = dataset[dataset['Sex'] == 'female']
male = dataset[dataset['Sex'] == 'male']

# plot female passenger
ax = sns.distplot(female[female['Survived'] == 1].Age.dropna(), 
                  bins=20, label=survived, ax=axes[0], kde=False )
ax = sns.distplot(female[female['Survived'] == 0].Age.dropna(), 
                  bins=20, label=not_survived, ax=axes[0], kde=False )

ax.legend()
ax.set_title('Female')

# plot male passenger
ax = sns.distplot(male[male['Survived'] == 1].Age.dropna(),
                 bins=20, label=survived, ax=axes[1], kde=False)
ax = sns.distplot(male[male['Survived'] == 0].Age.dropna(),
                 bins=20, label=not_survived, ax=axes[1], kde=False)

ax.legend()
ax.set_title('Male')

You will notice that male passengers have higher probability of survival when they are nearly 22 to 30 years old although passenger who not survived is still higher. For women, the survival chance are higher between 14 and 40. You also notice that passenger who less 5 years had higher survival rate in both `Sex` feature.
<br>
These two first charts confirm that one old code of conduct that sailors and captains follow in case of threatening situations: <b>"Women and children first !".</b>

In [ ]:
from IPython.display import Image
Image("img/women_childrent_first.jpg",height=700,width=700)

### `Pclass`, `Embarked` and `Sex`

In [ ]:
g = sns.FacetGrid(dataset, col='Pclass', size=5, aspect=1)
g.map(sns.pointplot, 'Embarked', 'Survived', 'Sex', palette='deep', orde=None, hue_order=None).add_legend()

`Pclass` seems have influence to survival rate. Based on plot above, we know that male passengers in `Pclass`=1 and `Pclass`=2 have higher survival rate than male passengers in `Pclass`=3. Although female passenger have higher chance to survive, only female passenger in `Pclass`=3 who have higher survival rate.
<br>
Men have a high survival probability of they are on port C, but a low probability if they are on port Q or S.

### `Pclass`, `Fare`

In [ ]:
sns.barplot(x='Pclass', y='Survived', data=dataset)

The upper-class passenger (`Pclass` = 1) were more likely to have survived

In [ ]:
g = sns.FacetGrid(dataset, col='Survived', hue='Pclass', size=6, aspect=1)
g.map(plt.scatter, 'Age','Fare', s=10).add_legend()

`Survived` rate seems not have correlation with `Fare`

## Wrangle data

First, we'll drop `PassengerId` and `Ticket` from the dataset since it doesn't contribute to a person probability.

In [ ]:
dataset = dataset.drop(['PassengerId', 'Ticket'], axis=1)
dataset.head()

### Missing data
As a reminder, we have to deal with Cabin (687), Age (177) and Embarked (2).

#### `Age`
We'll replace the null values with the median `Age` which is more accurate to handle missing values

In [ ]:
dataset['Age'].fillna(dataset['Age'].median(), inplace=True)
# convert Age values to int
dataset['Age'] = dataset['Age'].astype(int)
dataset.head()

In [ ]:
print('Number of missing value in Age: {}'.format(dataset['Age'].isnull().sum()))

#### `Embarked`
Since the `Embarked` feature has only 2 missing values, we'll just fill these with the most common values

In [ ]:
dataset['Embarked'].describe()

In [ ]:
common_value = 'S'
dataset['Embarked'] = dataset['Embarked'].fillna(common_value)
print('Number of missing value in Embarked: {}'.format(dataset['Embarked'].isnull().sum()))

### Creating new feature

### Title
we'll extract `Title` feature from `Name` feature using regular expressions (regex). The RegEx pattern (\w+\.) matches the first word which ends with a dot character within Name feature

In [ ]:
dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)
pd.crosstab(dataset['Title'], dataset['Sex'])

We get 17 title from extracting, but we'll reduce them all to Mrs, Miss, Mr and Master since it's most common title.

In [ ]:
def replace_titles(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col', 'Sir']:
        return 'Mr'
    elif title in ['Countess', 'Mme', 'Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [ ]:
dataset['Title'] = dataset.apply(replace_titles, axis=1)
dataset[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

Now we can safely drop the `Name` feature from dataset

In [ ]:
dataset = dataset.drop(['Name'], axis=1)
dataset.head()

### `Family Size`

`SibSp` and `Parch` would make more sense when combined as one feature that show the total number of family. We also create new feature that show if someone is alone or not.

In [ ]:
dataset['FamilySize'] = dataset['Parch'] + dataset['SibSp']

In [ ]:
def check_familysize(x):
    x = x['FamilySize']
    if x > 0:
        return 0
    else:
        return 1

In [ ]:
dataset['IsAlone'] = dataset.apply(check_familysize, axis=1)

In [ ]:
dataset = dataset.drop(['Parch', 'SibSp'], axis=1)
dataset.head()

#### Deck

In [ ]:
# Replace missing values with "Unknown"
dataset['Cabin'] = dataset['Cabin'].fillna('Unknown')

# Create a feature for the deck
dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())

In [ ]:
dataset[['Deck', 'Survived']].groupby(['Deck'], as_index=False).mean()

In [ ]:
pd.crosstab(dataset['Deck'], dataset['Survived'])

In [ ]:
dataset[(dataset['Pclass'] == 1) & (dataset['IsAlone'] == 1)]